In [18]:
!pip install --upgrade transformers

In [7]:
import os

os.environ["HF_TOKEN"] = ""


In [10]:


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-27b-it")
tokenizer("hello world i am meet")

{'input_ids': [2, 23391, 1902, 858, 1006, 2874], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [16]:
from google import genai

client = genai.Client(api_key="")

result = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents="meet is human. cat is animal and pet")

print(result.embeddings)
print(len(result.embeddings[0].values))



[ContentEmbedding(values=[-0.024769034, 0.0014970013, -0.0004949651, -0.07227179, -0.008343422, 0.0014104587, -0.008390712, 0.017661588, 0.029142234, 0.0024340188, -0.020047935, -0.028535362, -0.013904153, 0.01643324, 0.1286774, -0.0006196886, 0.0023797718, 0.010951217, -0.00856463, -0.033701662, -0.015137434, 0.013121961, 0.01612018, -0.018535947, -0.015530919, 0.013772314, 0.0005552358, 0.0038070055, 0.007934407, -0.0049561965, -0.0061168266, 0.03516298, -0.03427421, 0.02095206, 0.009991722, -0.032841146, 0.0142472675, -0.009275798, -0.018753238, 0.0022774711, 0.0042677796, 0.009519403, -0.0018988063, -0.020254081, -0.00019492462, -0.019132867, 0.00068048865, -0.005508663, 0.012075108, 0.013896672, 0.01213773, 0.022105852, -0.01023691, -0.14664224, 0.0023425699, 0.00016579485, -0.0071020266, -0.011968848, 0.0070497426, -0.0041171703, -0.025078768, 0.008314884, 0.0069402214, -0.0032153064, -0.02529708, -0.0075747613, 0.016808456, -0.008009803, 0.0046973447, 0.0012571824, 0.028130114, 

In [21]:

import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it", torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-27b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")

input_tokens = tokenizer("write a python code for adding 2 numbers", return_tensors="pt")
input_tokens


{'input_ids': tensor([[     2,   5986,    496,  23181,   3393,    573,   8009, 236743, 236778,
           4945]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [34]:
out = model(input_ids=input_tokens["input_ids"])
# only model gives all linear

print(out)

# all this are linears below

CausalLMOutputWithPast(loss=None, logits=tensor([[[-14.2266,  -1.1276,   4.6604,  ..., -14.7097, -14.7547, -14.6708],
         [-15.1654,  -3.5582,  -2.7689,  ..., -15.7508, -15.7075, -15.6461],
         [-13.0947,  -1.9868,  -4.7474,  ..., -14.8744, -14.8597, -14.7893],
         ...,
         [-11.1661,   1.1202,   1.0321,  ..., -12.3541, -12.2692, -12.3469],
         [-17.7597,  -2.8388,  -2.7308,  ..., -19.7018, -19.7166, -19.5911],
         [-16.3902,  -1.0508,  -1.1780,  ..., -18.8579, -18.9244, -18.7177]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.HybridCache object at 0x7c468935fa50>, hidden_states=None, attentions=None)


In [38]:
out = model.generate(input_ids=input_tokens["input_ids"],max_new_tokens = 200)
# model.generate will give linear with higest probability
# by default max tokens size is small so sometimes ouput may not complete so increase max tokens
print(out)

tensor([[     2,   5986,    496,  23181,   3393,    573,   8009, 236743, 236778,
           4945, 236761,    108,   2717,   6719,    107,   2063,   1138, 236779,
          34488, 236769, 236781, 236764,    570,   1473,    107,    138,  12234,
         174054,   1156,   4945,   3075, 236761,    108,    138,  13432, 236787,
            107,    140, 236781, 236787,    669,   1171,   1548, 236761,    107,
            140, 236762, 236787,    669,   1855,   1548, 236761,    108,    138,
          42652, 236787,    107,    140,    818,   2324,    529,   1123,    532,
            570, 236761,    107,    138,  12234,    107,    138,   2330,    578,
           1123,    900,    570,    107,    138,   2060,   2324,    108, 236865,
          14691,  14120,    107,   3744, 236770,    578, 236743, 236810,    107,
           3744, 236778,    578, 236743, 236800,    107,   3709,    578,   1138,
         236779,  34488, 236769,   3744, 236770, 236764,   1152, 236778, 236768,
            107,   1995, 236

In [39]:
# now detokenize
tokenizer.batch_decode(out)

['<bos>write a python code for adding 2 numbers.\n\n```python\ndef add_numbers(x, y):\n  """Adds two numbers together.\n\n  Args:\n    x: The first number.\n    y: The second number.\n\n  Returns:\n    The sum of x and y.\n  """\n  sum = x + y\n  return sum\n\n# Example usage\nnum1 = 5\nnum2 = 3\nresult = add_numbers(num1, num2)\nprint(result)  # Output: 8\n```\n\n**Explanation:**\n\n1.  **`def add_numbers(x, y):`**: This line defines a function named `add_numbers` that takes two arguments, `x` and `y`.\n2.  **`"""Adds two numbers together..."""`**: This is a docstring that explains what the function does.  It\'s good practice to include docstrings.\n3.  **`sum']